In [1]:
import numpy as np
import pandas as pd
import math
from joblib import dump, load

from BSP import *

Datapath = "../Data/"

In [2]:
df_train = pd.read_pickle("../Data/df_train.p")
df_test = pd.read_pickle("../Data/df_test.p")

In [3]:
m = load(Datapath+"fitted_m.joblib")

In [4]:
vals_test = df_test[['Temp', 'Rainfall']].values
X_test = np.array([list(vals_test[i]) + list(vals_test[i+1]) for i in range(len(vals_test)-1)])
# We can not predict for the final day as we do not have the weather prediction for the day after.
y_test = np.array(df_test['Demand'].values)[:-1]

In [5]:
df = df_test

In [6]:
i = 0
BSP = Non_stationary_BSP(df['Demand'][i], m_in=m)

In [56]:
i += 1
BSP.progress_day(df['Demand'][i], X_in=X_test[i-1])
print('t+1: ', BSP.t)
print('D[t-2]:', BSP.D[BSP.t-3])
print('mu: ', BSP.mu)
print('sigma: ', BSP.sigma)
print('I: ', BSP.I)
print('S: ', BSP.S)
print('Q: ', BSP.Q)
print('short: ', np.mean(BSP.short))
print('waste: ', np.mean(BSP.waste))
print('performance: ', BSP.performance())

t+1:  50
D[t-2]: 3
mu:  1.4453723907671276
sigma:  1.5018923255830143
I:  [1, 1, 1.0, 1.0, 1.0]
S:  4.0
Q:  [0, 1.0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 2.0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 2.0, 0, 0, 0, 0, 1.0, 2.0, 0, 2.0, 0, 0, 0, 0, 1.0, 2.0, 1.0, 0, 0, 0, 0, 2.0, 3.0, 0, 1.0, 0, 0, 0]
short:  0.10204081632653061
waste:  0.30612244897959184
performance:  (105.71428571428572, 0.6666666666666666, 0.2777777777777778)


class Non_stationary_BSP:
    D = []                                  # The real historical demand grouped by weekday, provided by the data, but only accessed after a day is over.
#     f = 1                                   # The fifo-lifo level, which should be 1 throughout this assignment.
    z = 2                                   # The safety factor, for this assignment it is set to 2.
    cw = 100                                # The cost over overstocking (waste) a product.
    cs = 500                                # The cost of being a product short.
    t = 1                                   # The current day starting at 1.
    S = 0                                   # The stock level, is always equal to mu + z*sigma.
    I_tot = 0                               # The total current inventory, sum over I.
    Q = []                                  # The order quantity for a certain day is equal to S-I_tot.
    mu = 0                                  # The predicted/expected demand for today. Can be set by moving average or some other function.
    sigma = 0                               # The predicted/expected standard deviation for today. Should be approximately sqrt(mu).
    mu_hist = []                            # The list of historical mu's grouped by weekday
    I = [0]*5                               # The inventory of the product, taking into account how long the product has been in the inventory
                                            # I[i] gives the inventory of product that has been in the inventory for i days.
    waste = []                              # The number of products wasted for each day.
    short = []                              # The number of products short for each day.
    avg_mu = 2.2211538461538463             # The average demand available in the data
    
    # The initialisation function leaves the BSP in the state it would be at the beginning of the day.
    def __init__(self, D_in, z_in=2, cw_in=100, cs_in=500, t_in=1, Q_in=[], mu_hist_in = [], sigma_hist_in = [], I_in=[0]*5, waste_in = [], short_in = [], avg_mu_in = 2.2211538461538463):
        self.D = [D_in]
#         self.f = f_in
        self.z = z_in
        self.cw = cw_in
        self.cs = cs_in
        self.t = t_in
        self.S = 0
        self.Q = Q_in
        self.mu_hist = mu_hist_in
        self.sigma_hist = sigma_hist_in
        self.I = I_in
        self.I_tot = sum(self.I)
        self.mu = 0
        self.sigma = 0
        self.waste = waste_in
        self.short = short_in
        self.avg_mu = avg_mu_in
    
    # At the beginning of the day, the order of yesterday is added to the inventory, 
    # each inventory item is shifted and the last items are thrown out.
    def update_inventory(self):
        self.waste.append(self.I[0])
        for i in range(len(self.I)-1):
            self.I[i] = self.I[i+1]
        if self.t == 1:
            self.I[-1] = 0
        else:
            self.I[-1] = self.Q[-1]
        self.I_tot = sum(self.I)

    # Given the demand of days t-14, t-13, t-7 and t-6, calculate the average moving weight prediction
    # for the today and tomorrow.
    def mv_avg_predict(self):
        if self.t==1:
            self.mu = self.avg_mu
        elif self.t<15:
            self.mu = self.D[self.t-2]
        else:
            d_14, d_13, d_7, d_6 = self.D[self.t-15], self.D[self.t-14], self.D[self.t-8], self.D[self.t-7]
            self.mu = (d_14+d_13+d_7+d_6)/2
    
    # The sigma is calculated according to what I presume to be the correct formula
    def calc_sigma(self):
        if self.t < 3:
            self.sigma = 0
        else:
            D = np.array([self.D[i] + self.D[i+1] for i in range(len(self.D[:self.t-2]))])
            mu_avg = np.mean(self.mu_hist)
            diff = D - mu_avg
#             print(diff.dot(diff))
            self.sigma = np.sqrt(diff.dot(diff)/(self.t-2))
    
    def process_demand(self):
        demand = self.D[-1]
        if demand >= self.I_tot:
            self.I = [0]*5
            self.short.append(demand-self.I_tot)
        else:
            i = 1
            while demand>0:
                supply = self.I[-i]
                self.I[-1] = max(0, demand)
                demand = max(0, demand-supply)
                i+=1
            self.short.append(0)
            
    def progress_day(self, daily_demand):
        # At the beginning of the day, we append the mu-history with yesterday's mu and sigma.
        self.mu_hist.append(self.mu)
        self.sigma_hist.append(self.sigma)
        # Next we update the inventory with yesterday's order which should have come in today.
        self.update_inventory()
        # We then predict the new mu and sigma for today.
        self.mv_avg_predict()
        self.calc_sigma()
        # The Stock level for tomorrow is determined, it is equal to mu + z*sigma
        self.S = round(self.mu + self.z*self.sigma)
        # The order quantity for tomorrow is equal to S minus the current inventory and the total order quantity is updated.
        self.Q.append(max(0, self.S-self.I_tot))
        # Next we let the customers buy stuff (as possible) and subtract the day's demand from our current inventory.
        self.process_demand()
        # We increase the day counter to reflect today.
        self.t+=1
        # Finally we add the demand for the next day.
        self.D.append(daily_demand)
        
    def performance(self, wup=14):
        waste, short, Q = self.waste[wup:], self.short[wup:], self.Q[wup:]
        avg_cost = np.mean(waste)*self.cw + np.mean(short)*self.cs
        w_perc = np.sum(waste)/np.sum(Q)
        s_perc = np.sum(short)/np.sum(Q)
        return avg_cost, w_perc, s_perc
        